모듈 임포트

In [ ]:
from bs4 import BeautifulSoup 

import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import time
import re
from tqdm import tqdm
from lxml import etree

크롬 드라이버 설치

In [ ]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

`빵집 이름`이 담긴 파일 가져오기
- `LISTLY` 사용하여 만든 엑셀 파일에서 이름만 따로 가져온다.
- 참고로 크롤링시 클릭이 먹히지 않기 때문에 해당 상호명을 검색했을 때, 원하는 지점 하나만 뜨도록 `빵집 이름`을 더 구체적으로 수정해준다. ex. 비파티세리->비파티세리 압구정
- 후에 만들어질 url 담을 컬럼도 만들어준다.

In [ ]:
df = pd.read_excel('서울시 마들렌 맛집] 리스트 (1018).xlsx') 
df['naver_map_url'] = ''
df = df.drop(columns=['Unnamed: 0'])
df

`빵집 이름` 이 담긴 컬럼의 이름을 "검색어"로 수정함.(엑셀)
1. 검색 url을 만들어 검색하고,
2. 검색된 플레이스의 페이지 url을 다시 가져온다.(검색 url의 가게코드 이용하는 원리로 추측됨.)
- 변수이름은 final_url. 
- 없는 경우 공백 

In [ ]:
for i, keyword in enumerate(df['검색어'].tolist()): 
    
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0]} 행", keyword) 
    
    try: 
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place' # 검색 url 만들기 
        driver.get(naver_map_search_url)
        time.sleep(5)

        cu = driver.current_url
        res_code = re.findall(r"place/(\d+)", cu)
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#' 
        
        print(final_url)
        df['naver_map_url'][i]=final_url 
        
    except IndexError: 
        df['naver_map_url'][i]= ''
        print('none')

### 각 매장별 리뷰 데이터프레임 만들기
리뷰 리스트 만들어서 데이터프레임으로 변환 후,
상호명 리스트(name_list)를 컬럼으로 지정함.
- 방법은 https://jenn1won.tistory.com/15 와 비슷하나,
- 필요하지 않은 데이터(작성자, 사진 등)은 코드에서 제거함.
- By , lxml 모듈에서 오류가 나서 따로 설치를 진행함.
- 셀렉터 값도 내 컴퓨터와 달라 수정해줌.  
- 더보기 버튼을 20번만 눌러서 약 200개 리뷰 수집
- 리뷰수가 다르면 데이터프레임 생성하는 데 오류 생겨서 리뷰수 200개로 잘라서 넣음.

In [ ]:
df_result=pd.DataFrame()

for i in range(len(df)): 
    review_list= []
    
    print('======================================================')
    
    # url 접속
    driver.get(df['naver_map_url'][i]) 
    thisurl = df['naver_map_url'][i]
    time.sleep(2) 
    
    # 더보기버튼 25번 클릭(한 번당 약 10개 뽑힌다. 넉넉히 25)
    num = 0
    while num < 25:
        try: 
            time.sleep(1) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(3) 
            
            driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a').send_keys(Keys.ENTER) 
            time.sleep(3) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) 
            time.sleep(1) 
            num += 1
        except : 'none'

    # 파싱
    html = driver.page_source 
    soup = BeautifulSoup(html, 'lxml') 
    time.sleep(1) 
    
    # 식당 이름, 구분, 총 개수 출력
    restaurant_name = df['검색어'][i]
    print('식당 이름 : '+restaurant_name)

    
    try: 
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'DJJvD'})[0].text 
    
    except: 
        restaurant_classificaton = 'none'
    
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try: 
        one_review = soup.find_all('div', attrs = {'class':'ZZ4OK IwhtZ'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수 
        print('리뷰 총 개수 : '+str(review_num)) 
        
        # 목표치인 리뷰 200개씩만 모으기
        for i in range(200):
            
            try: 
                review_content = one_review[i].find('span', attrs = {'class':'zPfVt'}).text
                review_list.append(review_content)
            except:
                review_content = "" 
            print('리뷰 내용 : '+review_content) 
            
      
    except NoSuchElementException: 
        none_review = "네이버 리뷰 없음" 
        print(none_review)
        review_num = 0 
        
    # 데이터 프레임 만들기
    df_result[f"{restaurant_name}"] = review_list.copy()

**결과 출력**

In [ ]:
df_result.to_excel('리뷰모음.xlsx')